20180634_최준혁_HW2

## Married Women's Labor Force Participation

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

plt.rc('font', family='Malgun Gothic')
plt.rcParams['axes.unicode_minus'] = False

In [2]:
data = pd.read_excel('mroz.xls')
df = data.copy()
df.head()

,inlf,hours,kidslt6,kidsge6,age,educ,wage,repwage,hushrs,husage,...,faminc,mtr,motheduc,fatheduc,unem,city,exper,nwifeinc,lwage,expersq
0,1,1610,1,0,32,12,3.3540,2.65,2708,34,...,16310,0.7215,12,7,5.0,0,14,10.910060,1.210154,196
1,1,1656,0,2,30,12,1.3889,2.65,2310,30,...,21800,0.6615,7,7,11.0,1,5,19.499981,0.328512,25
2,1,1980,1,3,35,12,4.5455,4.04,3072,40,...,21040,0.6915,12,7,5.0,0,15,12.039910,1.514138,225
3,1,456,0,3,34,12,1.0965,3.25,1920,53,...,7300,0.7815,7,7,5.0,0,6,6.799996,0.092123,36
4,1,1568,1,2,31,14,4.5918,3.60,2000,32,...,27300,0.6215,12,14,9.5,1,7,20.100058,1.524272,49


In [3]:
df.fillna(0, inplace=True) #wage의 결측치를 소득없음(0)으로 대체 : 전부 노동 비참여자

### report the OLS estimation result. what kind of standard errors would you use?

In [4]:
y = df[['inlf']]
X = df[['age','educ','exper','nwifeinc','kidslt6','kidsge6']]
X = sm.add_constant(X)

In [5]:
model = sm.OLS(y,X).fit()

- 종속변수가 binary하므로 이분산이 의심됨. 따라서 white test를 진행하겠음

In [6]:
from sklearn.linear_model import LinearRegression
from statsmodels.stats.diagnostic import het_white

In [7]:
white_test = het_white(model.resid,  model.model.exog)

#define labels to use for output of White's test
labels = ['Test Statistic', 'Test Statistic p-value', 'F-Statistic', 'F-Test p-value']

#print results of White's test
print(dict(zip(labels, white_test)))

{'Test Statistic': 111.20640491523122, 'Test Statistic p-value': 3.4696521734486925e-12, 'F-Statistic': 4.6527387194732475, 'F-Test p-value': 2.5788968539478646e-13}


- p-value가 매우낮으므로 동분산이라는 귀무가설을 기각함.
<br> -> 따라서 white robust standard error를 사용할 것이다

In [8]:
ols_robust=sm.OLS(y, X).fit(cov_type = "HC0")
ols_robust.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   inlf   R-squared:                       0.254
Model:                            OLS   Adj. R-squared:                  0.248
Method:                 Least Squares   F-statistic:                     66.17
Date:                Sun, 06 Nov 2022   Prob (F-statistic):           6.45e-66
Time:                        23:45:49   Log-Likelihood:                -429.12
No. Observations:                 753   AIC:                             872.2
Df Residuals:                     746   BIC:                             904.6
Df Model:                           6                                         
Covariance Type:                  HC0                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.7072      0.145      4.869      0.000       0.423       0.992
age           -0.0177      0.002     -7.829      0.000      -0.022      -0.013
educ           0.0398      0.007      5.551      0.000       0.026       0.054
exper          0.0226      0.002     10.799      0.000       0.018       0.027
nwifeinc      -0.0033      0.002     -2.213      0.027      -0.006      -0.000
kidslt6       -0.2718      0.031     -8.690      0.000      -0.333      -0.211
kidsge6        0.0125      0.014      0.926      0.355      -0.014       0.039
==============================================================================
Omnibus:                      193.244   Durbin-Watson:                   0.471
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               37.883
Skew:                          -0.184   Prob(JB):                     5.94e-09
Kurtosis:                       1.965   Cond. No.                         488.
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
"""

kidsge6변수는 유의한 변수가 아니고, nwifeinc은 5%수준에서 유의한 변수이다 

### what is the marginal effect of education for those whose education level is 12

In [9]:
df['educ_12'] = 0
df['educ_12'][df['educ']==12] = 1

In [10]:
df['educ_12'].value_counts()

1    381
0    372
Name: educ_12, dtype: int64

In [11]:
y = df[['inlf']]
X = df[['age','educ','exper','nwifeinc','kidslt6','kidsge6','educ_12']]
X = sm.add_constant(X)

In [12]:
ols_robust=sm.OLS(y, X).fit(cov_type = "HC0")
ols_robust.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   inlf   R-squared:                       0.255
Model:                            OLS   Adj. R-squared:                  0.248
Method:                 Least Squares   F-statistic:                     57.44
Date:                Sun, 06 Nov 2022   Prob (F-statistic):           9.12e-66
Time:                        23:45:49   Log-Likelihood:                -428.53
No. Observations:                 753   AIC:                             873.1
Df Residuals:                     745   BIC:                             910.1
Df Model:                           7                                         
Covariance Type:                  HC0                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.7405      0.150      4.922      0.000       0.446       1.035
age           -0.0178      0.002     -7.857      0.000      -0.022      -0.013
educ           0.0390      0.007      5.377      0.000       0.025       0.053
exper          0.0226      0.002     10.824      0.000       0.019       0.027
nwifeinc      -0.0034      0.001     -2.287      0.022      -0.006      -0.000
kidslt6       -0.2736      0.031     -8.749      0.000      -0.335      -0.212
kidsge6        0.0128      0.014      0.943      0.345      -0.014       0.039
educ_12       -0.0342      0.032     -1.074      0.283      -0.097       0.028
==============================================================================
Omnibus:                      189.385   Durbin-Watson:                   0.472
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               37.488
Skew:                          -0.181   Prob(JB):                     7.24e-09
Kurtosis:                       1.968   Cond. No.                         499.
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
"""

The marginal effect of education for those whose education level is 12 = 0.1035


### Run a probit regression. Does your answer to the previous question change

In [13]:
from statsmodels.discrete.discrete_model import Probit

In [14]:
probit_model = Probit(y, X).fit(cov_type = "HC0")
probit_model.summary()

Optimization terminated successfully.
         Current function value: 0.538752
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                          Probit Regression Results                           
==============================================================================
Dep. Variable:                   inlf   No. Observations:                  753
Model:                         Probit   Df Residuals:                      745
Method:                           MLE   Df Model:                            7
Date:                Sun, 06 Nov 2022   Pseudo R-squ.:                  0.2121
Time:                        23:45:49   Log-Likelihood:                -405.68
converged:                       True   LL-Null:                       -514.87
Covariance Type:                  HC0   LLR p-value:                 1.452e-43
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.6657      0.504      1.321      0.187      -0.322       1.653
age           -0.0557      0.008     -6.853      0.000      -0.072      -0.040
educ           0.1320      0.026      5.109      0.000       0.081       0.183
exper          0.0704      0.008      8.558      0.000       0.054       0.086
nwifeinc      -0.0119      0.005     -2.338      0.019      -0.022      -0.002
kidslt6       -0.8781      0.115     -7.634      0.000      -1.103      -0.653
kidsge6        0.0353      0.044      0.798      0.425      -0.051       0.122
educ_12       -0.1064      0.103     -1.036      0.300      -0.308       0.095
==============================================================================
"""

In [15]:
probit_model.get_margeff().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
       Probit Marginal Effects       
=====================================
Dep. Variable:                   inlf
Method:                          dydx
At:                           overall
==============================================================================
                dy/dx    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
age           -0.0170      0.002     -7.393      0.000      -0.021      -0.012
educ           0.0402      0.007      5.377      0.000       0.026       0.055
exper          0.0215      0.002     10.260      0.000       0.017       0.026
nwifeinc      -0.0036      0.002     -2.367      0.018      -0.007      -0.001
kidslt6       -0.2677      0.032     -8.434      0.000      -0.330      -0.206
kidsge6        0.0108      0.013      0.799      0.424      -0.016       0.037
educ_12       -0.0324      0.031     -1.038      0.299      -0.094       0.029
==============================================================================
"""

educ_12의 marginal effect는 변화가 없다.

### Calculate the average partial effect of education on the probability of labor force participation using both the linear probability model and the probit model

In [16]:
probit_model.get_margeff(at = 'overall').summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
       Probit Marginal Effects       
=====================================
Dep. Variable:                   inlf
Method:                          dydx
At:                           overall
==============================================================================
                dy/dx    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
age           -0.0170      0.002     -7.393      0.000      -0.021      -0.012
educ           0.0402      0.007      5.377      0.000       0.026       0.055
exper          0.0215      0.002     10.260      0.000       0.017       0.026
nwifeinc      -0.0036      0.002     -2.367      0.018      -0.007      -0.001
kidslt6       -0.2677      0.032     -8.434      0.000      -0.330      -0.206
kidsge6        0.0108      0.013      0.799      0.424      -0.016       0.037
educ_12       -0.0324      0.031     -1.038      0.299      -0.094       0.029
==============================================================================
"""

In [17]:
edu = list(df['educ'].value_counts().index)
edu

[12, 16, 14, 17, 10, 13, 11, 8, 9, 15, 7, 6, 5]

In [18]:
inlf_df = df[df['inlf']==1]

In [19]:
inlf_df[inlf_df['educ']==12]

,inlf,hours,kidslt6,kidsge6,age,educ,wage,repwage,hushrs,husage,...,mtr,motheduc,fatheduc,unem,city,exper,nwifeinc,lwage,expersq,educ_12
0,1,1610,1,0,32,12,3.3540,2.65,2708,34,...,0.7215,12,7,5.0,0,14,10.910060,1.210154,196,1
1,1,1656,0,2,30,12,1.3889,2.65,2310,30,...,0.6615,7,7,11.0,1,5,19.499981,0.328512,25,1
2,1,1980,1,3,35,12,4.5455,4.04,3072,40,...,0.6915,12,7,5.0,0,15,12.039910,1.514138,225,1
3,1,456,0,3,34,12,1.0965,3.25,1920,53,...,0.7815,7,7,5.0,0,6,6.799996,0.092123,36,1
5,1,2032,0,0,54,12,4.7421,4.70,1040,57,...,0.6915,14,7,7.5,1,33,9.859054,1.556480,1089,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
420,1,1944,0,2,49,12,2.5720,2.70,1920,53,...,0.7215,7,7,5.0,1,13,14.434031,0.944684,169,1
421,1,50,1,1,45,12,3.4600,5.00,1780,46,...,0.7215,7,7,9.5,1,9,23.709000,1.241269,81,1
424,1,2450,0,1,40,12,5.3061,6.50,2008,40,...,0.6215,7,7,5.0,1,21,22.641056,1.668857,441,1
426,1,1760,0,1,33,12,3.4091,3.21,3380,34,...,0.5815,12,16,11.0,1,14,23.999985,1.226448,196,1


In [20]:
sum = 0
for i in edu:
    sum += len(inlf_df[inlf_df['educ']==i])
sum/len(df)

0.5683930942895087

## Individual's Pension plan

### Report  the proportion of indivisuals in each catagory of the dependent variable. What is the proportion of individuals who most prefer stocks

In [21]:
data = pd.read_excel('pension.xls')
df = data.copy()
df.head()

,id,pyears,prftshr,choice,female,married,age,educ,finc25,finc35,finc50,finc75,finc100,finc101,wealth89,black,stckin89,irain89,pctstck
0,38,1.0,0,1,0,1,64,12,0,0,1,0,0,0,77.900002,0,1,1,0
1,152,6.0,1,1,1,1,56,13,0,0,0,1,0,0,154.899994,0,1,1,50
2,152,25.0,1,1,0,1,56,12,0,0,0,1,0,0,154.899994,0,1,1,50
3,182,20.0,1,0,1,1,63,12,1,0,0,0,0,0,232.500000,0,1,1,100
4,222,35.0,0,1,0,1,67,12,0,1,0,0,0,0,179.000000,0,0,1,100


In [22]:
df.fillna(0, inplace = True)

In [23]:
df_p = df[['pctstck']].value_counts()

In [24]:
df_p

pctstck
50         72
0          64
100        58
dtype: int64

In [25]:
print(df_p[0]/df_p.sum())
print(df_p[50]/df_p.sum())
print(df_p[100]/df_p.sum())

0.32989690721649484
0.3711340206185567
0.29896907216494845


In [26]:
print("The proportion of individuals who most prefer stocks :",df_p[100]/df_p.sum())

The proportion of individuals who most prefer stocks : 0.29896907216494845


### One can consider age, gender, martial status, race, education level, family income, and wealth as important factors that may effect the portfolio choice. Run an ordered probit regression and provide an interpretation on the estimated coefficient on the wealth variable

In [27]:
df.head()

,id,pyears,prftshr,choice,female,married,age,educ,finc25,finc35,finc50,finc75,finc100,finc101,wealth89,black,stckin89,irain89,pctstck
0,38,1.0,0,1,0,1,64,12,0,0,1,0,0,0,77.900002,0,1,1,0
1,152,6.0,1,1,1,1,56,13,0,0,0,1,0,0,154.899994,0,1,1,50
2,152,25.0,1,1,0,1,56,12,0,0,0,1,0,0,154.899994,0,1,1,50
3,182,20.0,1,0,1,1,63,12,1,0,0,0,0,0,232.500000,0,1,1,100
4,222,35.0,0,1,0,1,67,12,0,1,0,0,0,0,179.000000,0,0,1,100


In [28]:
y = df[['pctstck']]
X = df[['age','female','married','black','educ',
        'finc25','finc50','finc75','finc100','finc101','wealth89']]

In [29]:
from statsmodels.miscmodels.ordinal_model import OrderedModel
order_prob = OrderedModel(y,X,distr='probit').fit(method='bfgs')
order_prob.summary()

Optimization terminated successfully.
         Current function value: 1.061248
         Iterations: 48
         Function evaluations: 54
         Gradient evaluations: 54


<class 'statsmodels.iolib.summary.Summary'>
"""
                             OrderedModel Results                             
==============================================================================
Dep. Variable:                pctstck   Log-Likelihood:                -205.88
Model:                   OrderedModel   AIC:                             437.8
Method:            Maximum Likelihood   BIC:                             480.2
Date:                Sun, 06 Nov 2022                                         
Time:                        23:45:52                                         
No. Observations:                 194                                         
Df Residuals:                     181                                         
Df Model:                          13                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
age           -0.0499      0.022     -2.223      0.026      -0.094      -0.006
female         0.0095      0.202      0.047      0.962      -0.386       0.406
married        0.0824      0.231      0.357      0.721      -0.370       0.535
black         -0.0502      0.274     -0.183      0.854      -0.586       0.486
educ           0.0213      0.035      0.609      0.542      -0.047       0.090
finc25        -0.5036      0.257     -1.959      0.050      -1.007       0.000
finc50        -0.1642      0.246     -0.667      0.505      -0.646       0.318
finc75        -0.3350      0.311     -1.079      0.281      -0.944       0.274
finc100       -0.1493      0.306     -0.487      0.626      -0.749       0.451
finc101       -0.7503      0.392     -1.916      0.055      -1.518       0.017
wealth89      -0.0001      0.000     -0.364      0.716      -0.001       0.001
0/50          -3.4013      1.535     -2.216      0.027      -6.409      -0.394
50/100         0.0082      0.101      0.082      0.935      -0.190       0.206
==============================================================================
"""

wealth의 계수는 종속변수를 설명하는데 유의한 변수도 아니며 영향이 없다.

### choice로 다시 실행

In [30]:
y = df.choice
order_prob = OrderedModel(y,X,distr='probit').fit(method='bfgs')
order_prob.summary()

Optimization terminated successfully.
         Current function value: 0.599945
         Iterations: 54
         Function evaluations: 59
         Gradient evaluations: 59


<class 'statsmodels.iolib.summary.Summary'>
"""
                             OrderedModel Results                             
==============================================================================
Dep. Variable:                 choice   Log-Likelihood:                -116.39
Model:                   OrderedModel   AIC:                             256.8
Method:            Maximum Likelihood   BIC:                             296.0
Date:                Sun, 06 Nov 2022                                         
Time:                        23:45:54                                         
No. Observations:                 194                                         
Df Residuals:                     182                                         
Df Model:                          12                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
age           -0.0103      0.026     -0.401      0.688      -0.061       0.040
female        -0.1533      0.242     -0.633      0.527      -0.628       0.321
married       -0.3056      0.266     -1.147      0.251      -0.828       0.217
black         -0.5573      0.317     -1.760      0.078      -1.178       0.063
educ           0.0208      0.042      0.500      0.617      -0.061       0.102
finc25        -0.6190      0.287     -2.158      0.031      -1.181      -0.057
finc50        -0.1488      0.281     -0.529      0.597      -0.701       0.403
finc75         1.0272      0.437      2.349      0.019       0.170       1.884
finc100       -0.0646      0.354     -0.183      0.855      -0.758       0.628
finc101        0.2526      0.476      0.531      0.595      -0.680       1.185
wealth89      -0.0001      0.000     -0.303      0.762      -0.001       0.001
0/1           -1.1163      1.780     -0.627      0.531      -4.605       2.373
==============================================================================
"""

fin25와 fin75변수는 5%수준에서 유의한 변수이다. 가족들의 소득에 따라 choice가 바뀔 수 있다

### APE 계산

In [31]:
df_p = df[df['choice']==1]['pctstck'].value_counts()
term1 = df_p[100]/df_p.sum()

In [32]:
df_p = df[df['choice']==0]['pctstck'].value_counts()
term2 = df_p[100]/df_p.sum()

In [33]:
term1 - term2

0.03092436974789914

In [34]:
df['term1'] = (df['pctstck']== 100) & (df['choice'] ==1)
df['term2'] = (df['pctstck']== 100) & (df['choice'] ==0)

df['term1'][df['term1'] == True] = 1
df['term1'][df['term1'] == False] = 0
df['term2'][df['term2'] == True] = 1
df['term2'][df['term2'] == False] = 0

df['term3'] = df['term1'] - df['term2']

In [35]:
df['term3'].sum()/len(df)

0.08247422680412371